## Setup
Set up some imports that we need for recipes

In [1]:
import {
  derive,
  handler,
  JSONSchema,
  NAME,
  UI,
  recipe,
  Schema,
  cell,
  run,
  getDoc
} from "../../packages/cli/notebook_shim.ts";


## Most simple recipe
The entry point is to call "recipe" which returns an anonymous function.
You also pass it a function that is what your recipe "does". It takes in a list of input references, does work, and then outputs a result.
We'll go into all that but for now, we take in nothing, and give back nothing. 
You can see that this function object has an `argumentSchema` (which is empty) and also a `resultSchema` which is also empty.


In [2]:
recipe("super simple recipe", () => ({}));

[Function (anonymous)] {
  argumentSchema: {
    type: "object",
    properties: {},
    description: "super simple recipe"
  },
  resultSchema: {},
  initial: {},
  result: {},
  nodes: [],
  toJSON: [Function: toJSON]
}

## UI conventions
Most recipes will have a [NAME] and a [UI] property. We use this when we render the recipes.
You can see them under `result`

In [3]:
const helloRecipe = recipe("super simple recipe", () => ({
  [NAME]: "Hello World Recipe",
  [UI]: {
    tag: "os-container",
    children: ["Hello World!"]
  }
}));
helloRecipe

[Function (anonymous)] {
  argumentSchema: {
    type: "object",
    properties: {},
    description: "super simple recipe"
  },
  resultSchema: {},
  initial: {},
  result: {
    "$NAME": "Hello World Recipe",
    "$UI": { tag: "os-container", children: [ "Hello World!" ] }
  },
  nodes: [],
  toJSON: [Function: toJSON]
}

## ArgumentSchema

Now let's add an `ArgumentSchema`. We also need to include a `ResultSchema`, but it's just an empty stub for now — we'll come back to result schemas later.

We're focusing on the `ArgumentSchema` (also called the input schema). You can think of it like the parameters to a function. In this example, we’ll pass in one parameter: `name`.

To supply a value for the `name` field, we create a **reactive cell** using `getDoc(...).asCell()`. A cell is just a reactive value, the system tracks it and updates anything that depends on it when it changes.

Inside the recipe, we use `derive(name, fn)` to set up a reactive function. The function inside `derive` is automatically called whenever the value of `name` changes.

In this case, we’re using `console.log(...)` to prove that our input is wired up correctly. You should see "name changed" in the output logs when the recipe runs. That confirms we successfully passed a value into the `ArgumentSchema`, and that the reactive system responded.


In [4]:
// this is the input to the recipe
const ArgumentSchema = {
  type: "object",
  properties: {
    name: { type: "string" }
  },
  required: ["name"]
} as const satisfies JSONSchema;

// stubbed empty result schema
const ResultSchema = {
  type: "object",
  properties: {}
} as const satisfies JSONSchema;

// make the recipe, which is a function
const helloRecipe = recipe(ArgumentSchema, ResultSchema, ({ name }) => {
  // Create a derived value that reacts to changes in `name`
  derive(name, (nameValue) => {
    console.log("name changed", nameValue);
    return "hello";
  });

  // Return nothing for now
  return {};
});

// make input a proper cell for reactive programming
const input = getDoc("Ellyse", "name", "test").asCell();

// this is where the output will go, but we’re not using it yet
const resultDoc = getDoc(undefined, "run hello", "test");

const output = await run(helloRecipe, { name: input }, resultDoc);


generateRecipeId: generated id ba4jcagevpvlxuhv6i4awqcf67ycjmtamb3krwlfesl5nlvkedesrdcrb
name changed Ellyse


## ResultSchema

Now that we've seen how to define an `ArgumentSchema`, let's introduce the `ResultSchema`.

If `ArgumentSchema` defines the **inputs** to the recipe (like parameters to a function), then `ResultSchema` defines the **outputs** — the values this recipe will produce and expose.

In this example, we define a `ResultSchema` with a single field called `echoed`, which is a string. This schema defines the expected shape of the object returned from the function inside the `recipe(...)` definition.

Inside the recipe, we use `derive(...)` to create a reactive value called `echoed`, which updates whenever `name` changes. This value is then returned as part of the recipe's result.

In the end, we grab `echoed` from the recipe's result and print it to the screen.


In [5]:
const ArgumentSchema = {
  type: "object",
  properties: {
    name: { type: "string" }
  },
  required: ["name"]
} as const satisfies JSONSchema;

const ResultSchema = {
  type: "object",
  properties: {
    echoed: { type: "string" }
  },
  required: ["echoed"]
} as const satisfies JSONSchema;

const helloRecipe = recipe(ArgumentSchema, ResultSchema, ({ name }) => {
  // Create a derived value that reacts to changes in `name`
  const echoed = derive(name, (nameValue) => {
    console.log("name changed", nameValue);
    return `Hello, ${nameValue}!`;
  });

  return {
    [NAME]: "Hello World",
    echoed // included in `result` (see ResultSchema)
  };
});

// note here that resultDoc and output are the same documents because both hold the results of the recipe
const input = getDoc("Ellyse", "name", "test").asCell();
const resultDoc = getDoc(undefined, "run hello", "test");
const outputDoc = await run(helloRecipe, { name: input }, resultDoc);
const echoedValueFromOutput = outputDoc.getAsQueryResult(["echoed"])
console.log("got echoed value from the recipe output: ", echoedValueFromOutput)


generateRecipeId: generated id ba4jcb7hjgdcypwvoo33nudsc6r5ls543hxjfmcbwgkf5qynlsjm3apvs
name changed Ellyse
got echoed value from the recipe output:  Hello, Ellyse!
